In [ ]:
import stats as st
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Set global font size
plt.rcParams.update({"font.size": 24})  # General font size for ticks, legends, etc.

# Set specific font size for axis labels and title
plt.rcParams["axes.labelsize"] = 24  # Font size for axis labels (x and y axes)
plt.rcParams["axes.titlesize"] = 24  # Font size for plot title
plt.rcParams["xtick.labelsize"] = 20  # Font size for x-axis ticks
plt.rcParams["ytick.labelsize"] = 20  # Font size for y-axis ticks

### Read data

In [ ]:
import importlib

importlib.reload(st)

gaps_cp_df = st.parse_champsim_output(f"../results/GAPS/cp-data")
gaps_wp_df = st.parse_champsim_output(f"../results/GAPS/wp-data")
gaps_wpa_df = st.parse_champsim_output(f"../results/GAPS/wpa-data")

lcf_cp_df = st.parse_champsim_output(f"../results/LCF/cp-data")
lcf_wp_df = st.parse_champsim_output(f"../results/LCF/wp-data")
lcf_wpa_df = st.parse_champsim_output(f"../results/LCF/wpa-data")

spec_cp_df = st.merge_simpoints(st.parse_champsim_output(f"../results/SPEC/cp-data"))
spec_wp_df = st.merge_simpoints(st.parse_champsim_output(f"../results/SPEC/wp-data"))
spec_wpa_df = st.merge_simpoints(st.parse_champsim_output(f"../results/SPEC/wpa-data"))

### Group data

In [ ]:
groupings = {
    # Default config
    "default": "default",
    # L1I Prefetchers
    "l1i-next_line": "l1i-next_line",
    "l1i-barsa": "l1i-barsa",
    "l1i-bip": "l1i-bip",
    "l1i-djolt": "l1i-djolt",
    "l1i-epi": "l1i-epi",
    "l1i-fnlmma": "l1i-fnlmma_new",
    "l1i-mana": "l1i-mana",
    "l1i-pips": "l1i-pips",
    "l1i-tap": "l1i-tap",
    # L1D Prefetchers
    "l1d-next_line": "l1d-next_line",
    "l1d-ip_stride": "l1d-ip_stride",
    "l1d-berti": "l1d-berti",
    "l1d-ipcp": "l1d-ipcp",
    # L2C Prefetchers
    "l2c-next_line": "l2c-next_line",
    "l2c-ip_stride": "l2c-ip_stride",
    "l2c-spp": "l2c-spp_dev",
    "l2c-ampm": "l2c-ampm",
    "l2c-bingo": "l2c-bingo",
    "l2c-bop": "l2c-bop",
    "l2c-dspatch": "l2c-dspatch",
    "l2c-ipcp": "l2c-ipcp",
    "l2c-mlop": "l2c-mlop",
    "l2c-sandbox": "l2c-sandbox",
    "l2c-scooby": "l2c-scooby",
    "l2c-sms": "l2c-sms",
    "l2c-spp_ppf": "l2c-spp_ppf_dev",
    "l2c-streamer": "l2c-streamer",
    # LLC Prefetchers
    "llc-next_line": "llc-next_line",
    "llc-ip_stride": "llc-ip_stride",
    # LLC Replacement Policies
    "llc-ship": "ship",
    "llc-srrip": "srrip",
    "llc-drrip": "drrip",
    "llc-mockingjay": "mockingjay",
}

datasets = {
    "gaps": {"cp": gaps_cp_df, "wp": gaps_wp_df, "wpa": gaps_wpa_df},
    "lcf": {"cp": lcf_cp_df, "wp": lcf_wp_df, "wpa": lcf_wpa_df},
    "spec": {"cp": spec_cp_df, "wp": spec_wp_df, "wpa": spec_wpa_df},
}

results = {}

for label, group_by in groupings.items():
    for category, versions in datasets.items():
        for version, df in versions.items():
            key = f"{category}_{version}_{label}_df"
            grouped = st.group_by(df, group_by)
            results[key] = st.calculate_means(grouped)

### Instruction Overhead & MPKI (Fig. 5)

In [ ]:
suites = ["lcf", "gaps", "spec"]
colors = {"ED-WP": "steelblue", "TI-WP": "indianred"}

all_benchmarks = []
wp_speedups = []
wpa_speedups = []
suite_boundaries = []
suite_labels = []
mpki_values_lcf = []
mpki_values_gaps = []
mpki_values_spec = []

x_position = 0

for suite in suites:
    wp = results[f"{suite}_wp_default_df"]["wrong_path_insts_executed"]
    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    wp = wp.drop("gmean", errors="ignore")
    benchmarks = list(wp.index.drop(["gmean", "amean"], errors="ignore"))

    mpki = mpki.drop("gmean", errors="ignore")
    benchmarks = list(mpki.index.drop(["gmean", "amean"], errors="ignore"))

    # Create list of tuples of (benchmark, mpki)
    benchmark_mpki_pairs = list(zip(benchmarks, mpki))

    # Sort the benchmarks by MPKI value (ascending order for MPKI)
    benchmark_mpki_pairs.sort(key=lambda x: x[1])  # Sort ascending by MPKI

    # Extract sorted benchmarks and MPKI values
    sorted_benchmarks = [bm for bm, _ in benchmark_mpki_pairs]
    sorted_mpki_values = [mpki_val for _, mpki_val in benchmark_mpki_pairs]

    # Remove "amean" if it was still in any list
    if "amean" in sorted_benchmarks:
        amean_index = sorted_benchmarks.index("amean")
        sorted_mpki_values.pop(amean_index)  # Remove MPKI value for "amean"
        sorted_benchmarks.pop(amean_index)  # Remove "amean" from benchmark list

    benchmarks_labeled = [f"{suite.upper()}_{bm}" for bm in sorted_benchmarks]
    all_benchmarks.extend(benchmarks_labeled)

    # Update wp_speedups according to sorted benchmarks
    wp_speedups.extend(wp[sorted_benchmarks] / 100_000_000 * 100)

    # Store MPKI values for each suite
    if suite == "lcf":
        mpki_values_lcf.extend(sorted_mpki_values)
    elif suite == "gaps":
        mpki_values_gaps.extend(sorted_mpki_values)
    elif suite == "spec":
        mpki_values_spec.extend(sorted_mpki_values)

    # Save mid-point and boundary for annotations
    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)


fig = go.Figure()

fig.add_trace(
    go.Bar(x=all_benchmarks, y=wp_speedups, name="ED-WP", marker_color=colors["ED-WP"])
)

fig.add_trace(
    go.Scatter(
        x=all_benchmarks[: len(mpki_values_lcf)],
        y=mpki_values_lcf,
        mode="lines+markers",
        name="MPKI LCF",
        line=dict(color="red"),
        yaxis="y2",
    )
)

fig.add_trace(
    go.Scatter(
        x=all_benchmarks[
            len(mpki_values_lcf) : len(mpki_values_lcf) + len(mpki_values_gaps)
        ],
        y=mpki_values_gaps,
        mode="lines+markers",
        name="MPKI GAPS",
        line=dict(color="green"),
        yaxis="y2",
    )
)

fig.add_trace(
    go.Scatter(
        x=all_benchmarks[len(mpki_values_lcf) + len(mpki_values_gaps) :],
        y=mpki_values_spec,
        mode="lines+markers",
        name="MPKI SPEC",
        line=dict(color="blue"),
        yaxis="y2",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite annotations above
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # title="WP and WPA Speedup vs CP (Default)",
    xaxis=dict(
        # title=dict(
        #     text="Benchmark",
        #     standoff=50,  # Add vertical space between axis and title
        # ),
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,  # Ensure horizontal grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    yaxis=dict(
        title="Extra Instructions Executed (%)",
        # range=[-5, 40],  # max(max(wp_speedups), max(wpa_speedups)) * 1.1
        showgrid=True,  # Ensure vertical grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
        range=[0, 250],  # Adjusted range for better visibility
    ),
    yaxis2=dict(
        title="MPKI",
        overlaying="y",
        side="right",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=False,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dash",
        range=[0, 40],
        dtick=8,
    ),
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
    showlegend=False,
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("overhead_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("overhead_plot.pdf", format="pdf", width=1100, height=400, scale=2)

### L1I Hits & Misses (Fig. 6)

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
hits_diffs = []
misses_diffs = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load data
    cp_hits = results[f"{suite}_cp_default_df"]["L1I_TOTAL_HITS"]
    wp_hits = results[f"{suite}_wp_default_df"]["L1I_TOTAL_HITS"]
    cp_misses = results[f"{suite}_cp_default_df"]["L1I_TOTAL_MISS"]
    wp_misses = results[f"{suite}_wp_default_df"]["L1I_TOTAL_MISS"]
    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean
    for metric in [cp_hits, wp_hits, cp_misses, wp_misses, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list excluding amean
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x])

    # Add suite-prefixed amean at the end if available
    if "amean" in mpki.index:
        benchmarks_sorted.append(f"{suite.upper()}_amean")

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in benchmarks_sorted
    ]

    all_benchmarks.extend(labeled_benchmarks)

    # Get diffs in the same order
    hits_diff = ((wp_hits - cp_hits) / cp_hits) * 100
    misses_diff = ((wp_misses - cp_misses) / cp_misses) * 100

    hits_diffs.extend(
        [
            hits_diff.get(bm if "amean" not in bm else "amean")
            for bm in benchmarks_sorted
        ]
    )
    misses_diffs.extend(
        [
            misses_diff.get(bm if "amean" not in bm else "amean")
            for bm in benchmarks_sorted
        ]
    )

    # Annotations and dividers
    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Create Plot
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=hits_diffs,
        name="L1I Hits",
        marker_color="mediumseagreen",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=misses_diffs,
        name="L1I Misses",
        marker_color="salmon",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 300],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",  # Vertical layout
        x=0.025,  # Left side
        y=0.95,  # Top side
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",  # Optional: semi-transparent white background
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.add_annotation(
    x=18 - 0.25,
    y=300,
    text=f"{hits_diffs[18]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=19 - 0.25,
    y=300,
    text=f"{hits_diffs[19]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=20 - 0.25,
    y=300,
    text=f"{hits_diffs[20]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=21 - 0.25,
    y=300,
    text=f"{hits_diffs[21]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=22 - 0.25,
    y=300,
    text=f"{hits_diffs[22]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=27 + 0.25,
    y=300,
    text=f"{misses_diffs[27]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)

fig.add_annotation(
    x=29 + 0.25,
    y=300,
    text=f"{misses_diffs[29]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)


fig.show()


# Save the figure as an interactive HTML file
fig.write_html("l1i_hits_misses_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image(
    "l1i_hits_misses_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### L1D, L2C, LLC Misses (Fig. 7)

In [ ]:
levels = ["L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

# For each suite, process the data
for suite in suites:
    # Load MPKI data and drop "gmean" from the indices
    mpki = results[f"{suite}_cp_default_df"]["MPKI"]
    benchmarks = list(mpki.index.drop("gmean", errors="ignore"))

    # Sort the benchmarks except 'amean'
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x], reverse=False)

    # Explicitly move 'amean' to the end
    if "amean" in benchmarks_sorted:
        benchmarks_sorted.remove("amean")
    benchmarks_sorted.append("amean")

    # Label benchmarks, appending the suite prefix
    labeled_benchmarks = [f"{suite.upper()}_{bm}" for bm in benchmarks_sorted]
    all_benchmarks.extend(labeled_benchmarks)

    # For each level, compute misses diffs
    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][f"{level}_TOTAL_MISS"]
        wp_misses = results[f"{suite}_wp_default_df"][f"{level}_TOTAL_MISS"]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")  # Ensure we drop gmean
        misses_diffs_by_level[level].extend(diff[benchmarks_sorted])

    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Plot
fig = go.Figure()

colors = {"L1D": "orange", "L2C": "dodgerblue", "LLC": "mediumorchid"}

for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level} Misses",
            marker_color=colors[level],
        )
    )

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase in Misses (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 90],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",
        x=0.025,
        y=0.95,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

# Add annotation for a specific benchmark if necessary
fig.add_annotation(
    x=12 + 0.33,
    y=90,
    text=f"{misses_diffs_by_level['LLC'][12]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)

fig.show()

# Save output
fig.write_html("misses_L1D_L2C_LLC.html", include_plotlyjs="cdn")
fig.write_image("misses_L1D_L2C_LLC.pdf", format="pdf", width=1100, height=400, scale=2)

### L1D, L2C, LLC Hits (Fig. 8)

In [ ]:
levels = ["L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]
hits_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    # Get the benchmarks and drop "gmean" from the indices
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    # Sort the benchmarks by MPKI (or any metric you're interested in)
    mpki = results[f"{suite}_cp_default_df"]["MPKI"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x], reverse=False)

    # Explicitly move 'amean' to the end
    if "amean" in benchmarks_sorted:
        benchmarks_sorted.remove("amean")
    benchmarks_sorted.append("amean")

    # Label the benchmarks, adding the suite prefix except for 'amean'
    labeled_benchmarks = [f"{suite.upper()}_{bm}" for bm in benchmarks_sorted]
    all_benchmarks.extend(labeled_benchmarks)

    # For each level, calculate the difference in hits
    for level in levels:
        cp_hits = results[f"{suite}_cp_default_df"][f"{level}_TOTAL_HITS"]
        wp_hits = results[f"{suite}_wp_default_df"][f"{level}_TOTAL_HITS"]
        diff = ((wp_hits - cp_hits) / cp_hits) * 100
        diff = diff.drop("gmean", errors="ignore")  # Drop gmean from the diff
        hits_diffs_by_level[level].extend(diff[benchmarks_sorted])

    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Plotting
fig = go.Figure()

colors = {"L1D": "orange", "L2C": "dodgerblue", "LLC": "mediumorchid"}

# Add the bars for each level
for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=hits_diffs_by_level[level],
            name=f"{level} Hits",
            marker_color=colors[level],
        )
    )

# Add the suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add the suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

# Layout settings for better visibility
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase in Hits (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-20, 90],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",
        x=0.025,
        y=0.95,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

# Annotations for specific values (optional, based on your needs)
fig.add_annotation(
    x=12 + (0.33 / 2.0),
    y=90,
    text=f"{hits_diffs_by_level['L2C'][12]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)

fig.add_annotation(
    x=12 + 0.33,
    y=10,
    text=f"{hits_diffs_by_level['LLC'][12]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-75,
)

fig.add_annotation(
    x=20 - (0.33 / 2.0),
    y=90,
    text=f"{hits_diffs_by_level['L1D'][20]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)

fig.show()

# Save the plot
fig.write_html("hits_L1D_L2C_LLC.html", include_plotlyjs="cdn")
fig.write_image("hits_L1D_L2C_LLC.pdf", format="pdf", width=1100, height=400, scale=2)

### L1I, L1D, L2C, LLC CP (Fig. 9)

In [ ]:
levels = ["L1I", "L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    # Get the benchmarks and drop "gmean" from the indices
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    # Sort the benchmarks by MPKI (or any metric you're interested in)
    mpki = results[f"{suite}_cp_default_df"]["MPKI"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x], reverse=False)

    # Explicitly move 'amean' to the end
    if "amean" in benchmarks_sorted:
        benchmarks_sorted.remove("amean")
    benchmarks_sorted.append("amean")

    # Label the benchmarks, adding the suite prefix except for 'amean'
    labeled_benchmarks = [f"{suite.upper()}_{bm}" for bm in benchmarks_sorted]
    all_benchmarks.extend(labeled_benchmarks)

    # For each level, calculate the difference in misses
    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][f"{level}_POLLUTION_CP_MISS"]
        wp_misses = results[f"{suite}_wp_default_df"][f"{level}_POLLUTION_CP_MISS"]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")  # Drop gmean from the diff
        misses_diffs_by_level[level].extend(diff[benchmarks_sorted])

    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Plotting
fig = go.Figure()

colors = {"L1I": "green", "L1D": "orange", "L2C": "dodgerblue", "LLC": "mediumorchid"}

# Add the bars for each level
for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level}",
            marker_color=colors[level],
        )
    )

# Add the suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add the suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

# Layout settings for better visibility
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="No-WP Miss Reduction (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-100, 50],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.025,
        y=0.15,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save the plot
fig.write_html("L1I_L1D_L2C_LLC.html", include_plotlyjs="cdn")
fig.write_image("L1I_L1D_L2C_LLC.pdf", format="pdf", width=1100, height=400, scale=2)

### Cycle breakdown (Fig. 10)

In [ ]:
# total cycles
lcf_total_cycles = results["lcf_wp_default_df"]["total_cycles"]
gaps_total_cycles = results["gaps_wp_default_df"]["total_cycles"]
spec_total_cycles = results["spec_wp_default_df"]["total_cycles"]

# wp_cycles
lcf_wp_cycles = results["lcf_wp_default_df"]["wp_cycles"]
gaps_wp_cycles = results["gaps_wp_default_df"]["wp_cycles"]
spec_wp_cycles = results["spec_wp_default_df"]["wp_cycles"]

# wp_not_available_cycles
lcf_wp_not_available_cycles = results["lcf_wp_default_df"]["WP_Not_Available_Cycles"]
gaps_wp_not_available_cycles = results["gaps_wp_default_df"]["WP_Not_Available_Cycles"]
spec_wp_not_available_cycles = results["spec_wp_default_df"]["WP_Not_Available_Cycles"]

# resteer events
lcf_resteer_events = results["lcf_wp_default_df"]["Resteer Events"] * 12
gaps_resteer_events = results["gaps_wp_default_df"]["Resteer Events"] * 12
spec_resteer_events = results["spec_wp_default_df"]["Resteer Events"] * 12

# cp_cyles
lcf_cp_cycles = (
    lcf_total_cycles - lcf_wp_cycles - lcf_wp_not_available_cycles - lcf_resteer_events
)
gaps_cp_cycles = (
    gaps_total_cycles
    - gaps_wp_cycles
    - gaps_wp_not_available_cycles
    - gaps_resteer_events
)
spec_cp_cycles = (
    spec_total_cycles
    - spec_wp_cycles
    - spec_wp_not_available_cycles
    - spec_resteer_events
)

In [ ]:
import plotly.graph_objects as go
import pandas as pd


def process_cycles(
    total_cycles, wp_cycles, wp_na_cycles, wp_resteer_events, suite_name
):
    df = (
        total_cycles.to_frame("total_cycles")
        .join(wp_cycles.rename("wp_cycles"))
        .join(wp_na_cycles.rename("wp_na_cycles"))
        .join(wp_resteer_events.rename("resteer_events"))
    )

    # Normalize index to catch and drop geomean variations
    cleaned_index = df.index.str.strip().str.lower()
    df.index = cleaned_index

    # Drop any row where index is 'gmean'
    df = df[~(df.index == "gmean")]

    # Drop any row where index is 'amean'
    df = df[~(df.index == "amean")]

    # Get the MPKI values
    mpki_values = results[f"{suite_name.lower()}_wp_default_df"]["MPKI"]

    # Drop any row where index is 'gmean'
    mpki_values = mpki_values[~(mpki_values.index == "gmean")]
    # Drop any row where index is 'amean'
    mpki_values = mpki_values[~(mpki_values.index == "amean")]

    # Sort the DataFrame by MPKI values
    df = df.loc[mpki_values.index]
    df = df.join(mpki_values.rename("MPKI"))
    df = df.sort_values(by="MPKI", ascending=True)
    df = df.drop(columns=["MPKI"])
    df.index.name = "Benchmark"

    # Compute cp_cycles
    df["cp_cycles"] = (
        df["total_cycles"] - df["wp_cycles"] - df["wp_na_cycles"] - df["resteer_events"]
    )

    return df


# Process and combine
lcf_df = process_cycles(
    lcf_total_cycles,
    lcf_wp_cycles,
    lcf_wp_not_available_cycles,
    lcf_resteer_events,
    "LCF",
)
gaps_df = process_cycles(
    gaps_total_cycles,
    gaps_wp_cycles,
    gaps_wp_not_available_cycles,
    gaps_resteer_events,
    "GAPS",
)
spec_df = process_cycles(
    spec_total_cycles,
    spec_wp_cycles,
    spec_wp_not_available_cycles,
    spec_resteer_events,
    "SPEC",
)

full_df = pd.concat(
    [
        lcf_df[["cp_cycles", "wp_cycles", "wp_na_cycles", "resteer_events"]],
        gaps_df[["cp_cycles", "wp_cycles", "wp_na_cycles", "resteer_events"]],
        spec_df[["cp_cycles", "wp_cycles", "wp_na_cycles", "resteer_events"]],
    ]
)

# calculate the mean of the columns
mean_values = full_df.mean(axis=0)

# add the mean values to the full_df
full_df.loc["amean"] = mean_values

# Normalize to percentages
percent_df = full_df.div(full_df.sum(axis=1), axis=0) * 100

# Plot
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=percent_df.index,
        y=percent_df["cp_cycles"],
        name="CP",
        marker_color="indianred",
    )
)
fig.add_trace(
    go.Bar(
        x=percent_df.index,
        y=percent_df["wp_cycles"],
        name="ED-WP",
        marker_color="mediumseagreen",
    )
)
fig.add_trace(
    go.Bar(
        x=percent_df.index,
        y=percent_df["wp_na_cycles"],
        name="ED-WP Not Available",
        marker_color="steelblue",
    )
)
fig.add_trace(
    go.Bar(
        x=percent_df.index,
        y=percent_df["resteer_events"],
        name="Resteer",
        marker_color="gold",
    )
)

# Add vertical lines between suites
suite_labels = ["LCF", "GAPS", "SPEC"]  # Adjust if there are more suites
boundary_indices = [
    len(lcf_df),
    len(lcf_df) + len(gaps_df),
    len(lcf_df) + len(gaps_df) + len(spec_df),
]  # indices separating the suites

for i in boundary_indices:
    fig.add_vline(
        x=i - 0.5,  # Place the line between the suite groups
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig.update_layout(
    barmode="stack",
    # title="Cycle Breakdown per Benchmark (Percentage)",
    xaxis_title="Benchmark",
    yaxis_title="Percentage of Total Cycles",
    legend_title="Cycle Type",
    xaxis_tickangle=-45,
    yaxis=dict(ticksuffix="%", range=[0, 100]),
    template="plotly_white",
    margin=dict(t=60, b=100),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("cycle_breakdown_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image(
    "cycle_breakdown_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### ROB (Fig. 11)

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
rob_full_diff = []  # List for storing differences
cp_rob_at_miss_all = []
wp_rob_at_miss_all = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load metrics
    cp_rob_at_miss = results[f"{suite}_cp_default_df"][
        "Average_ROB_Occupancy_at_Mispredict"
    ]
    wp_rob_at_miss = results[f"{suite}_wp_default_df"][
        "Average_ROB_Occupancy_at_Mispredict"
    ]

    cp_rob_full = results[f"{suite}_cp_default_df"]["ROB_Full_Events"]
    wp_rob_full = results[f"{suite}_wp_default_df"]["ROB_Full_Events"]

    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean from all metrics
    for metric in [cp_rob_at_miss, wp_rob_at_miss, cp_rob_full, wp_rob_full, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list (exclude amean)
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    sorted_benchmarks = sorted(benchmarks, key=lambda x: mpki[x])

    # Collect metric values *in the sorted order*
    cp_vals = [cp_rob_at_miss[bm] for bm in sorted_benchmarks]
    wp_vals = [wp_rob_at_miss[bm] for bm in sorted_benchmarks]
    rob_diff_vals = [
        ((wp_rob_full[bm] - cp_rob_full[bm]) / cp_rob_full[bm]) * 100
        for bm in sorted_benchmarks
    ]

    # Add amean if available
    if "amean" in mpki.index:
        amean_label = f"{suite.upper()}_amean"
        sorted_benchmarks.append(amean_label)
        cp_vals.append(cp_rob_at_miss["amean"])
        wp_vals.append(wp_rob_at_miss["amean"])
        rob_diff_vals.append(
            ((wp_rob_full["amean"] - cp_rob_full["amean"]) / cp_rob_full["amean"]) * 100
        )

    # Label benchmarks
    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in sorted_benchmarks
    ]

    # Extend global lists
    all_benchmarks.extend(labeled_benchmarks)
    cp_rob_at_miss_all.extend(cp_vals)
    wp_rob_at_miss_all.extend(wp_vals)
    rob_full_diff.extend(rob_diff_vals)

    # Labels and dividers
    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)


# Create Plot
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=cp_rob_at_miss_all,
        name="No-WP",
        marker_color="mediumseagreen",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=wp_rob_at_miss_all,
        name="ED-WP",
        marker_color="salmon",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    barmode="group",
    legend=dict(
        orientation="v",  # Vertical layout
        x=0.025,  # Left side
        y=0.95,  # Top side
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",  # Optional: semi-transparent white background
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("rob_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("rob_plot.pdf", format="pdf", width=1100, height=400, scale=2)

### Speedup for all benchmarks (Fig. 12)

In [ ]:
suites = ["lcf", "gaps", "spec"]
wp_label = "ED-WP"
wpa_label = "TI-WP"
colors = {"ED-WP": "steelblue", "TI-WP": "indianred"}

all_benchmarks = []
wp_speedups = []
wpa_speedups = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    cp = results[f"{suite}_cp_default_df"]["IPC"]
    wp = results[f"{suite}_wp_default_df"]["IPC"]
    wpa = results[f"{suite}_wpa_default_df"]["IPC"]
    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean early
    cp_no_gmean = cp.drop("amean", errors="ignore")
    wp_no_gmean = wp.drop("amean", errors="ignore")
    wpa_no_gmean = wpa.drop("amean", errors="ignore")
    mpki_no_gmean = mpki.drop("amean", errors="ignore")

    # Sort benchmarks by MPKI
    sorted_benchmarks = mpki_no_gmean.sort_values().index.tolist()

    # Add gmean if present
    # has_gmean = "gmean" in cp.index and "gmean" in wp.index and "gmean" in wpa.index
    # if has_gmean:
    #     sorted_benchmarks.append("gmean")

    # Apply label (prefix gmean as well)
    benchmarks_labeled = [f"{suite.upper()}_{bm}" for bm in sorted_benchmarks]
    all_benchmarks.extend(benchmarks_labeled)

    # Extend data in the sorted order
    wp_speedups.extend(
        ((wp.reindex(sorted_benchmarks) / cp.reindex(sorted_benchmarks)) - 1) * 100
    )
    wpa_speedups.extend(
        ((wpa.reindex(sorted_benchmarks) / cp.reindex(sorted_benchmarks)) - 1) * 100
    )

    # Save annotation and boundaries
    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks, y=wpa_speedups, name=wpa_label, marker_color=colors["TI-WP"]
    )
)

fig.add_trace(
    go.Bar(x=all_benchmarks, y=wp_speedups, name=wp_label, marker_color=colors["ED-WP"])
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite annotations above
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # title="WP and WPA Speedup vs CP (Default)",
    xaxis=dict(
        # title=dict(
        #     text="Benchmark",
        #     standoff=50,  # Add vertical space between axis and title
        # ),
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,  # Ensure horizontal grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    yaxis=dict(
        title="Speedup (%)",
        range=[-5, 40],  # max(max(wp_speedups), max(wpa_speedups)) * 1.1
        showgrid=True,  # Ensure vertical grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
)

ccsv_index = all_benchmarks.index("GAPS_ccsv")
ccsv_wp_value = wp_speedups[ccsv_index]

fig.add_annotation(
    x=ccsv_index + 0.25,
    y=40,
    text=f"{ccsv_wp_value:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig.add_shape(
    type="line",
    x0=0,
    x1=1,
    y0=40,  # Match the y-axis max
    y1=40,
    xref="paper",  # Use the whole width of the plot
    yref="y",
    line=dict(color="black", width=2, dash="solid"),  # You can use 'dot', 'dash', etc.
    layer="above",
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("speedup_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("speedup_plot.pdf", format="pdf", width=1100, height=400, scale=2)

### L1I CP v WP (Fig. 13)

In [ ]:
from plotly import graph_objects as go

suites = ["lcf"]

pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

colorblind_colors = [
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
    "#CC79A7",
    "#999999",
    "#000000",
]

prefetchers = {
    "l1i-barsa": {"label": "BARÇA", "color": colorblind_colors[0]},
    "l1i-bip": {"label": "BIP", "color": colorblind_colors[1]},
    "l1i-djolt": {"label": "Djolt", "color": colorblind_colors[2]},
    "l1i-epi": {"label": "EIP", "color": colorblind_colors[3]},
    "l1i-fnlmma": {"label": "FNL+MMA", "color": colorblind_colors[4]},
    "l1i-mana": {"label": "Mana", "color": colorblind_colors[5]},
    "l1i-next_line": {"label": "Next Line", "color": colorblind_colors[6]},
    "l1i-pips": {"label": "PIPS", "color": colorblind_colors[7]},
    "l1i-tap": {"label": "TAP", "color": colorblind_colors[8]},
}

# Build MPKI-based order from a reference group (e.g., CP)
mpki_values = results["lcf_wp_default_df"].get("MPKI", {})
sorted_prefetchers = sorted(
    [pf for pf in prefetchers if pf in results["lcf_wp_default_df"]],
    key=lambda pf: mpki_values.get(pf, float("inf")),
)

# Append 'amean' if it's present
if "amean" in results["lcf_wp_default_df"].get("IPC", {}):
    sorted_prefetchers.append("amean")

suite_labels = ["Correct Path", "Wrong Path"]
speedups_by_prefetcher = {
    pf: {"cp": [], "wp": []} for pf in prefetchers
}  # Ensure CP, WP are initialized

for suite in suites:
    # Loop through CP, WP
    for group in ["cp", "wp"]:
        default_baseline = (
            results.get(f"{suite}_{group}_default_df", {})
            .get("IPC", {})
            .get("gmean", None)
        )
        if default_baseline is None:
            print(
                f"Warning: {suite}_{group}_default_df data is missing or incorrectly formatted."
            )
            continue

        for pf in prefetchers:
            p_pf = (
                results.get(f"{suite}_{group}_{pf}_df", {})
                .get("IPC", {})
                .get("gmean", None)
            )
            if p_pf is None:
                print(
                    f"Warning: {suite}_{group}_{pf}_df data is missing or incorrectly formatted."
                )
                continue

            speedup = ((p_pf / default_baseline) - 1) * 100
            speedups_by_prefetcher[pf][group].append(speedup)

# --- Create the first figure for Correct Path ---
fig_cp = go.Figure()

# Plot for Correct Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_cp.add_trace(
        go.Bar(
            x=[props["label"]],  # Prefetcher labels as x-axis
            y=[speedups_by_prefetcher[pf]["cp"][0]],  # Correct Path speedup
            name=props["label"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            marker_color=props["color"],
        )
    )

# Update layout for Correct Path figure
fig_cp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-2, 7],  # Adjust range for the first row if needed
    ),
    barmode="group",  # Group bars within each row
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.15,
        xanchor="center",
        x=0.5,
        font=dict(size=16),
    ),
    showlegend=False,
)

# Add annotations for Correct Path
fig_cp.add_annotation(
    x="EIP",
    y=7,
    text=f"{speedups_by_prefetcher['l1i-epi']['cp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.add_annotation(
    x="BIP",
    y=0,
    text=f"{speedups_by_prefetcher['l1i-bip']['cp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the first figure (Correct Path)
fig_cp.show()

# Save the first figure as an interactive HTML file
fig_cp.write_html("l1i_prefetchers_cp_plot.html", include_plotlyjs="cdn")

# Save the first figure as a static PDF
fig_cp.write_image(
    "l1i_prefetchers_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)


# --- Create the second figure for Wrong Path ---
fig_wp = go.Figure()

# Plot for Wrong Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_wp.add_trace(
        go.Bar(
            x=[props["label"]],  # Prefetcher labels as x-axis
            y=[speedups_by_prefetcher[pf]["wp"][0]],  # Wrong Path speedup
            name=props["label"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            marker_color=props["color"],
        )
    )

# Update layout for Wrong Path figure
fig_wp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-2, 7],  # Adjust range for the second row if needed
    ),
    barmode="group",  # Group bars within each row
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.15,
        xanchor="center",
        x=0.5,
        font=dict(size=16),
    ),
    showlegend=False,
)

# Add annotations for Wrong Path
fig_wp.add_annotation(
    x="BIP",
    y=0,
    text=f"{speedups_by_prefetcher['l1i-bip']['wp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.add_annotation(
    x="TAP",
    y=0,
    text=f"{speedups_by_prefetcher['l1i-tap']['wp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the second figure (Wrong Path)
fig_wp.show()

# Save the second figure as an interactive HTML file
fig_wp.write_html("l1i_prefetchers_wp_plot.html", include_plotlyjs="cdn")

# Save the second figure as a static PDF
fig_wp.write_image(
    "l1i_prefetchers_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### L1D CP vs WP (Fig. 14)

In [ ]:
from plotly import graph_objects as go

# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]
colorblind_colors = ["#99C945", "#CC61B0", "#24796C", "#DAA51B"]

# Prefetchers dictionary
prefetchers = {
    "l1d-berti": {"label": "Berti", "color": colorblind_colors[0]},
    "l1d-ip_stride": {"label": "IP Stride", "color": colorblind_colors[1]},
    "l1d-ipcp": {"label": "IPCP", "color": colorblind_colors[2]},
    "l1d-next_line": {"label": "Next Line", "color": colorblind_colors[3]},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]
suite_labels[1] = "GAP"  # Special case for "gaps"

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# --- Create the first figure for Correct Path ---
fig_cp = go.Figure()

# Plot for Correct Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_cp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        )
    )

    fig_cp.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

    fig_cp.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )


# Update layout for Correct Path figure
fig_cp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-7, 7],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

fig_cp.add_annotation(
    x=1 - 0.29,
    y=7,
    text=f"{speedups_by_prefetcher['l1d-berti']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.add_annotation(
    x=1 + 0.30,
    y=0,
    text=f"{speedups_by_prefetcher['l1d-next_line']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the first figure (Correct Path)
fig_cp.show()

# Save the first figure as an interactive HTML file
fig_cp.write_html("l1d_prefetchers_cp_plot.html", include_plotlyjs="cdn")

# Save the first figure as a static PDF
fig_cp.write_image(
    "l1d_prefetchers_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)


# --- Create the second figure for Wrong Path ---
fig_wp = go.Figure()

# Plot for Wrong Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_wp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        )
    )

    fig_wp.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

    fig_wp.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )


# Update layout for Wrong Path figure
fig_wp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-7, 7],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

fig_wp.add_annotation(
    x=1 - 0.29,
    y=6.3,
    text=f"{speedups_by_prefetcher['l1d-berti']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.add_annotation(
    x=1 + 0.30,
    y=0,
    text=f"{speedups_by_prefetcher['l1d-next_line']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the second figure (Wrong Path)
fig_wp.show()

# Save the second figure as an interactive HTML file
fig_wp.write_html("l1d_prefetchers_wp_plot.html", include_plotlyjs="cdn")

# Save the second figure as a static PDF
fig_wp.write_image(
    "l1d_prefetchers_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### L2C CP v WP (Fig. 15)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

colorblind_13_colors = [
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # bluish green
    "#F0E442",  # yellow
    "#0072B2",  # blue
    "#D55E00",  # vermillion
    "#CC79A7",  # reddish purple
    "#999999",  # gray
    "#000000",  # black
    "#009E73",  # green
    "#F0E442",  # yellow
    "#D55E00",  # red
    "#56B4E9",  # light blue
]

# Prefetchers dictionary
prefetchers = {
    "l2c-bingo": {"label": "Bingo", "color": colorblind_13_colors[0]},
    "l2c-ip_stride": {"label": "IP Stride", "color": colorblind_13_colors[3]},
    "l2c-mlop": {"label": "MLOP", "color": colorblind_13_colors[5]},
    "l2c-next_line": {"label": "Next Line", "color": colorblind_13_colors[6]},
    "l2c-sms": {"label": "SMS", "color": colorblind_13_colors[9]},
    "l2c-spp": {"label": "SPP", "color": colorblind_13_colors[10]},
    "l2c-streamer": {"label": "Streamer", "color": colorblind_13_colors[12]},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]
suite_labels[1] = "GAP"  # Special case for "gaps"

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# Plot for CP speedup (first figure)
fig_cp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_cp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        )
    )

# Add vertical lines to CP plot
for i in range(1, len(suite_labels)):
    fig_cp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_cp.update_layout(
    # title="CP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-8, 8],
        dtick=2,
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

fig_cp.add_annotation(
    x=1 - 0.34,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-bingo']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.add_annotation(
    x=1,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-next_line']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.show()

# Save the figure as an interactive HTML file for CP
fig_cp.write_html("l2c_prefetchers_cp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF for CP
fig_cp.write_image(
    "l2c_prefetchers_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

# Plot for WP speedup (second figure)
fig_wp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_wp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        )
    )

# Add vertical lines to WP plot
for i in range(1, len(suite_labels)):
    fig_wp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_wp.update_layout(
    # title="WP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-8, 8],
        dtick=2,
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

fig_wp.add_annotation(
    x=1 - 0.34,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-bingo']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.add_annotation(
    x=1,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-next_line']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.show()

# Save the figure as an interactive HTML file for WP
fig_wp.write_html("l2c_prefetchers_wp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF for WP
fig_wp.write_image(
    "l2c_prefetchers_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### LLC Repl CP v WP (Fig. 16)

In [ ]:
import plotly.graph_objects as go

# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

# Prefetchers dictionary
prefetchers = {
    "llc-ship": {"label": "SHIP", "color": "blue"},
    "llc-srrip": {"label": "SRRIP", "color": "green"},
    "llc-drrip": {"label": "DRRIP", "color": "orange"},
    "llc-mockingjay": {"label": "Mockingjay", "color": "purple"},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]
suite_labels[1] = "GAP"  # Special case for "gaps"

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# CP Speedup plot
fig_cp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_cp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        )
    )

# Add vertical lines to CP plot
for i in range(1, len(suite_labels)):
    fig_cp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_cp.update_layout(
    # title="CP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 10],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

# Show the plot for CP
fig_cp.show()

# Save the figure as an interactive HTML file
fig_cp.write_html("llc_cp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig_cp.write_image("llc_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2)

# WP Speedup plot
fig_wp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_wp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        )
    )

# Add vertical lines to WP plot
for i in range(1, len(suite_labels)):
    fig_wp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_wp.update_layout(
    # title="WP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 10],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

# Show the plot for WP
fig_wp.show()

# Save the figure as an interactive HTML file
fig_wp.write_html("llc_wp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig_wp.write_image("llc_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2)

# WIP

### L1I & L1D footprint

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
all_l1i_cp = []
all_l1i_wp = []
all_l1d_cp = []
all_l1d_wp = []
misses_diffs = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load data
    if suite != "lcf":
        l1i_cp = results[f"{suite}_cp_default_df"]["instr_foot_print"] * 64 / 1024
        l1i_wp = results[f"{suite}_wp_default_df"]["instr_foot_print"] * 64 / 1024
        l1d_cp = results[f"{suite}_cp_default_df"]["data_foot_print"] * 64 / 1024
        l1d_wp = results[f"{suite}_wp_default_df"]["data_foot_print"] * 64 / 1024
    else:
        l1i_cp = results[f"{suite}_cp_default_df"]["instr_foot_print"] * 64 / 1024**2
        l1i_wp = results[f"{suite}_wp_default_df"]["instr_foot_print"] * 64 / 1024**2
        l1d_cp = results[f"{suite}_cp_default_df"]["data_foot_print"] * 64 / 1024**2
        l1d_wp = results[f"{suite}_wp_default_df"]["data_foot_print"] * 64 / 1024**2

    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean
    for metric in [l1i_cp, l1i_wp, l1d_cp, l1d_wp, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list excluding amean
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x])

    # Add suite-prefixed amean at the end if available
    if "amean" in mpki.index:
        benchmarks_sorted.append(f"{suite.upper()}_amean")

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in benchmarks_sorted
    ]

    all_benchmarks.extend(labeled_benchmarks)

    all_l1i_cp.extend(
        [l1i_cp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )
    all_l1i_wp.extend(
        [l1i_wp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )
    all_l1d_cp.extend(
        [l1d_cp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )
    all_l1d_wp.extend(
        [l1d_wp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )

    # all_l1i_cp.extend(l1i_cp)
    # all_l1i_wp.extend(l1i_wp)
    # all_l1d_cp.extend(l1d_cp)
    # all_l1d_wp.extend(l1d_wp)

    # Annotations and dividers
    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Create Plot
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1i_cp,
        name="L1I No-WP",
        marker_color="mediumseagreen",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1i_wp,
        name="L1I ED-WP",
        marker_color="salmon",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1d_cp,
        name="L1D No-WP",
        marker_color="lightblue",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1d_wp,
        name="L1D ED-WP",
        marker_color="lightcoral",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 300],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",  # Vertical layout
        x=0.025,  # Left side
        y=0.95,  # Top side
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",  # Optional: semi-transparent white background
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()


# Save the figure as an interactive HTML file
fig.write_html("footprint.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("footprint.pdf", format="pdf", width=1100, height=400, scale=2)

### AMAT L1I, L2C, LLC, and ITLB

In [ ]:
levels = ["L1I", "L2C", "LLC", "ITLB"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm in ["amean", "gmean"] else bm for bm in benchmarks
    ]
    all_benchmarks.extend(labeled_benchmarks)

    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][
            f"{level}_AVERAGE_CP_INSTR_MISS_LATENCY"
        ]
        wp_misses = results[f"{suite}_wp_default_df"][
            f"{level}_AVERAGE_CP_INSTR_MISS_LATENCY"
        ]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")
        misses_diffs_by_level[level].extend(diff[benchmarks])

    suite_labels.append((x_position + len(benchmarks) / 2, suite.upper()))
    x_position += len(benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Plot
fig = go.Figure()

colors = {
    "L1I": "green",
    "L1D": "orange",
    "L2C": "dodgerblue",
    "LLC": "mediumorchid",
    "ITLB": "red",
}

for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level}",
            marker_color=colors[level],
        )
    )

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="CP Miss Reduction (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-100, 50],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.025,
        y=0.15,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save output
fig.write_html("AMAT_L1I_L2C_LLC_ITLB.html", include_plotlyjs="cdn")
fig.write_image(
    "AMAT_L1I_L2C_LLC_ITLB.pdf", format="pdf", width=1100, height=400, scale=2
)

In [ ]:
levels = ["L1D", "L2C", "LLC", "DTLB"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm in ["amean", "gmean"] else bm for bm in benchmarks
    ]
    all_benchmarks.extend(labeled_benchmarks)

    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][
            f"{level}_AVERAGE_CP_DATA_MISS_LATENCY"
        ]
        wp_misses = results[f"{suite}_wp_default_df"][
            f"{level}_AVERAGE_CP_DATA_MISS_LATENCY"
        ]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")
        misses_diffs_by_level[level].extend(diff[benchmarks])

    suite_labels.append((x_position + len(benchmarks) / 2, suite.upper()))
    x_position += len(benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Plot
fig = go.Figure()

colors = {
    "L1D": "green",
    "L1D": "orange",
    "L2C": "dodgerblue",
    "LLC": "mediumorchid",
    "DTLB": "red",
}

for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level}",
            marker_color=colors[level],
        )
    )

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="CP Miss Reduction (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-100, 50],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.025,
        y=0.15,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save output
fig.write_html("AMAT_L1D_L2C_LLC_DTLB.html", include_plotlyjs="cdn")
fig.write_image(
    "AMAT_L1D_L2C_LLC_DTLB.pdf", format="pdf", width=1100, height=400, scale=2
)

# TO DROP

### LLC CP v WP | Droped

In [ ]:
# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

# Prefetchers dictionary
prefetchers = {
    "llc-ip_stride": {"label": "IP Stride", "color": "#E69F00"},
    "llc-next_line": {"label": "Next Line", "color": "#56B4E9"},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# Create the subplot figure with 2 rows and 1 column
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.3,
    # subplot_titles=["CP Speedup (GAPS, SPEC, LCF)", "WP Speedup (GAPS, LCF)"]
)

# Plot for CP (Top row)
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        ),
        row=1,
        col=1,
    )

    fig.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=1,  # First row
        col=1,  # First column
    )

    fig.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=1,  # First row
        col=1,  # First column
    )


# Plot for WP (Bottom row)
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        ),
        row=2,
        col=1,
    )

    fig.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=2,  # First row
        col=1,  # First column
    )

    fig.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=2,  # First row
        col=1,  # First column
    )


# Update layout and styling
fig.update_layout(
    xaxis=dict(
        # title="Suite",
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - NO-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-10, 10],
    ),
    yaxis2=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-10, 10],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

fig.add_annotation(
    x=1 + 0.2,
    y=0,
    text=f"{speedups_by_prefetcher['llc-next_line']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig.add_annotation(
    x=1 + 0.2,
    y=0,
    text=f"{speedups_by_prefetcher['llc-next_line']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
    row=2,
    col=1,
)

# Show the plot
fig.show()

# Save the figure as an interactive HTML file
fig.write_html("llc_prefetchers_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image(
    "llc_prefetchers_plot.pdf", format="pdf", width=1100, height=800, scale=2
)

In [ ]:
import plotly.graph_objects as go

# Only one level: L1I, L1D
levels = ["L1I", "L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]

# Initialize containers to store useful and useless diffs for each benchmark
useful_l1i_diffs = []
useless_l1i_diffs = []
useful_l1d_diffs = []
useless_l1d_diffs = []
useful_l2c_diffs = []
useless_l2c_diffs = []
useful_llc_diffs = []
useless_llc_diffs = []
all_benchmarks = []
suite_boundaries = []  # List to store the boundary positions for vertical lines

# Collect data
x_position = 0  # To track the x-axis position for the boundaries

for suite in suites:
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    for bm in benchmarks:
        all_benchmarks.append(bm)

        # Collect useful and useless diffs for L1I
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "L1I_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "L1I_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_l1i_diffs.append(useful_diff)
        useless_l1i_diffs.append(useless_diff)

        # Collect useful and useless diffs for L1D
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "L1D_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "L1D_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_l1d_diffs.append(useful_diff)
        useless_l1d_diffs.append(useless_diff)

        # Collect useful and useless diffs for L2C
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "L2C_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "L2C_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_l2c_diffs.append(useful_diff)
        useless_l2c_diffs.append(useless_diff)

        # Collect useful and useless diffs for LLC
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "LLC_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "LLC_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_llc_diffs.append(useful_diff)
        useless_llc_diffs.append(useless_diff)

    # Store the boundary position after the last benchmark of the suite
    suite_labels.append((x_position + len(benchmarks) / 2, suite.upper()))
    x_position += len(benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Create x-axis labels for each benchmark
x_labels = all_benchmarks

# Plot
fig = go.Figure()

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

# Colors for the "useful" and "useless" parts of the bar
colors = (
    "steelblue",
    "lightsteelblue",
    "orange",
    "lightcoral",
    "mediumseagreen",
    "lightgreen",
    "mediumorchid",
    "plum",
)

# Create a trace for the "useless" and "useful" values for L1I (swap order)
fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_l1i_diffs,  # Useless comes first (on top)
        name="L1I Useless",
        marker_color=colors[1],  # Color for useless
        legendgroup="L1I Useless",
        offsetgroup="L1I",  # Group L1I bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_l1i_diffs,  # Useful comes second (on bottom)
        name="L1I Useful",
        marker_color=colors[0],  # Color for useful
        legendgroup="L1I Useful",
        offsetgroup="L1I",  # Group L1I bars together
    ),
)

# Second stack for L1D (useless on top and useful on bottom)
fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_l1d_diffs,  # Useless comes first (on top)
        name="L1D Useless",
        marker_color=colors[3],  # Color for L1D Useless
        legendgroup="L1D Useless",
        offsetgroup="L1D",  # Group L1D bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_l1d_diffs,  # Useful comes second (on bottom)
        name="L1D Useful",
        marker_color=colors[2],  # Color for L1D Useful
        legendgroup="L1D Useful",
        offsetgroup="L1D",  # Group L1D bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_l2c_diffs,  # Useless comes first (on top)
        name="L2C Useless",
        marker_color=colors[5],  # Color for L2C Useless
        legendgroup="L2C Useless",
        offsetgroup="L2C",  # Group L2C bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_l2c_diffs,  # Useful comes second (on bottom)
        name="L2C Useful",
        marker_color=colors[4],  # Color for L2C Useful
        legendgroup="L2C Useful",
        offsetgroup="L2C",  # Group L2C bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_llc_diffs,  # Useless comes first (on top)
        name="LLC Useless",
        marker_color=colors[7],  # Color for LLC Useless
        legendgroup="LLC Useless",
        offsetgroup="LLC",  # Group LLC bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_llc_diffs,  # Useful comes second (on bottom)
        name="LLC Useful",
        marker_color=colors[6],  # Color for LLC Useful
        legendgroup="LLC Useful",
        offsetgroup="LLC",  # Group LLC bars together
    ),
)

# Update the layout
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(x_labels))),
        ticktext=x_labels,  # Single label per benchmark
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Change in Prefetch Count (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    barmode="relative",  # Ensure the bars are grouped side by side
    legend=dict(
        orientation="h",
        x=0.025,
        y=1.05,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save the figure
# fig.write_html("stacked_per_level_L1I_L1D_prefetch_with_boundaries.html", include_plotlyjs="cdn")
# fig.write_image("stacked_per_level_L1I_L1D_prefetch_with_boundaries.pdf", format="pdf", width=1200, height=400, scale=2)

In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Find the indexes of the amean in all_benchmarks
amean_indexes = [i for i, bm in enumerate(all_benchmarks) if bm == "amean"]

# Filter out the amean indexes from the lists
f_useless_l1i_diffs = [
    diff for i, diff in enumerate(useless_l1i_diffs) if i not in amean_indexes
]
f_useful_l1i_diffs = [
    diff for i, diff in enumerate(useful_l1i_diffs) if i not in amean_indexes
]
f_useless_l1d_diffs = [
    diff for i, diff in enumerate(useless_l1d_diffs) if i not in amean_indexes
]
f_useful_l1d_diffs = [
    diff for i, diff in enumerate(useful_l1d_diffs) if i not in amean_indexes
]
f_useless_l2c_diffs = [
    diff for i, diff in enumerate(useless_l2c_diffs) if i not in amean_indexes
]
f_useful_l2c_diffs = [
    diff for i, diff in enumerate(useful_l2c_diffs) if i not in amean_indexes
]
f_useless_llc_diffs = [
    diff for i, diff in enumerate(useless_llc_diffs) if i not in amean_indexes
]
f_useful_llc_diffs = [
    diff for i, diff in enumerate(useful_llc_diffs) if i not in amean_indexes
]
f_all_benchmarks = [bm for i, bm in enumerate(all_benchmarks) if i not in amean_indexes]

df = pd.DataFrame(
    dict(
        year=f_all_benchmarks,
        var1=f_useless_l1i_diffs,
        var2=f_useful_l1i_diffs,
        var3=f_useless_l1d_diffs,
        var4=f_useful_l1d_diffs,
        var5=f_useless_l2c_diffs,
        var6=f_useful_l2c_diffs,
        var7=f_useless_llc_diffs,
        var8=f_useful_llc_diffs,
    )
)

fig = go.Figure()

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary * 4, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
    yaxis=dict(title_text="ED-WP Useful vs Useless Prefetches", showgrid=True),
    barmode="stack",
    legend=dict(
        orientation="h",  # horizontal layout
        yanchor="bottom",
        y=1.02,  # just above the plot area
        xanchor="center",
        x=0.5,
    ),
    xaxis=dict(
        title="Benchmarks",
        tickvals=list(range(len(f_all_benchmarks))),
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
)

groups = ["var1", "var2", "var3", "var4", "var5", "var6", "var7", "var8"]
colors = ["blue", "red", "green", "purple", "orange", "pink", "brown", "gray"]
names = [
    "L1I Useless",
    "L1I Useful",
    "L1D Useless",
    "L1D Useful",
    "L2C Useless",
    "L2C Useful",
    "LLC Useless",
    "LLC Useful",
]


i = 0
for r, n, c in zip(groups, names, colors):
    ## put var1 and var2 together on the first subgrouped bar
    if i <= 1:
        fig.add_trace(
            go.Bar(x=[df.year, [" "] * len(df.year)], y=df[r], name=n, marker_color=c),
        )
    ## put var3 and var4 together on the first subgrouped bar
    elif i <= 3 and i > 1:
        fig.add_trace(
            go.Bar(x=[df.year, ["  "] * len(df.year)], y=df[r], name=n, marker_color=c),
        )
    ## put var5 and var6 together on the first subgrouped bar
    elif i <= 5 and i > 3:
        fig.add_trace(
            go.Bar(
                x=[df.year, ["   "] * len(df.year)], y=df[r], name=n, marker_color=c
            ),
        )
    ## put var7 and var8 together on the first subgrouped bar
    elif i <= 7 and i > 5:
        fig.add_trace(
            go.Bar(
                x=[df.year, ["    "] * len(df.year)], y=df[r], name=n, marker_color=c
            ),
        )
    i += 1

fig.show()

# Save the figure as an interactive HTML file
fig.write_html(
    "stacked_per_level_.html",
    include_plotlyjs="cdn",
)

# Save the figure as a static PDF
fig.write_image(
    "stacked_per_level.pdf",
    format="pdf",
    width=1200,
    height=400,
    scale=2,
)

### WIP

In [ ]:
def plot_cp_l1i_prefetcher_speedup_geomean(results):
    suites = ["gaps", "spec", "lcf"]
    prefetchers = {
        "l1i-barsa": {"label": "Barsa", "color": "blue"},
        "l1i-bip": {"label": "BIP", "color": "green"},
        "l1i-djolt": {"label": "Djolt", "color": "orange"},
        "l1i-epi": {"label": "EPI", "color": "purple"},
        "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
        "l1i-mana": {"label": "Mana", "color": "magenta"},
        "l1i-next_line": {"label": "Next Line", "color": "yellow"},
        "l1i-pips": {"label": "PIPS", "color": "lime"},
        "l1i-tap": {"label": "TAP", "color": "brown"},
    }

    suite_labels = [s.upper() for s in suites]
    speedups_by_prefetcher = {pf: [] for pf in prefetchers}

    for suite in suites:
        cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
        for pf in prefetchers:
            cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
            speedup = ((cp_pf / cp_default) - 1) * 100
            speedups_by_prefetcher[pf].append(speedup)

    fig = go.Figure()

    for pf, props in prefetchers.items():
        fig.add_trace(
            go.Bar(
                x=suite_labels,
                y=speedups_by_prefetcher[pf],
                name=props["label"],
                marker_color=props["color"],
            )
        )

    fig.update_layout(
        xaxis=dict(
            title=dict(text="Suite", standoff=40),
            tickangle=0,
            showgrid=False,
        ),
        yaxis=dict(
            title="Speedup (%)",
            range=[-5, 40],
            showgrid=True,
            gridcolor="lightgray",
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor="black",
            zerolinewidth=2,
            griddash="dot",
        ),
        barmode="group",
        legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig.show()
    return fig


# Save the plot if needed
fig = plot_cp_l1i_prefetcher_speedup_geomean(results)
# fig.write_image("cp_l1i_prefetchers_speedup.svg", format="svg", width=1100, height=300)

In [ ]:
def plot_cp_l1i_prefetcher_speedup_geomean(results):
    suites = ["gaps", "spec", "lcf"]
    prefetchers = {
        "l1i-barsa": {"label": "Barsa", "color": "blue"},
        "l1i-bip": {"label": "BIP", "color": "green"},
        "l1i-djolt": {"label": "Djolt", "color": "orange"},
        "l1i-epi": {"label": "EPI", "color": "purple"},
        "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
        "l1i-mana": {"label": "Mana", "color": "magenta"},
        "l1i-next_line": {"label": "Next Line", "color": "yellow"},
        "l1i-pips": {"label": "PIPS", "color": "lime"},
        "l1i-tap": {"label": "TAP", "color": "brown"},
    }

    suite_labels = [s.upper() for s in suites]
    speedups_by_prefetcher = {pf: [] for pf in prefetchers}

    for suite in suites:
        cp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
        for pf in prefetchers:
            cp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
            speedup = ((cp_pf / cp_default) - 1) * 100
            speedups_by_prefetcher[pf].append(speedup)

    fig = go.Figure()

    for pf, props in prefetchers.items():
        fig.add_trace(
            go.Bar(
                x=suite_labels,
                y=speedups_by_prefetcher[pf],
                name=props["label"],
                marker_color=props["color"],
            )
        )

    fig.update_layout(
        xaxis=dict(
            title=dict(text="Suite", standoff=40),
            tickangle=0,
            showgrid=False,
        ),
        yaxis=dict(
            title="Speedup (%)",
            range=[-5, 40],
            showgrid=True,
            gridcolor="lightgray",
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor="black",
            zerolinewidth=2,
            griddash="dot",
        ),
        barmode="group",
        legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig.show()
    return fig


# Save the plot if needed
fig = plot_cp_l1i_prefetcher_speedup_geomean(results)
# fig.write_image("cp_l1i_prefetchers_speedup.svg", format="svg", width=1100, height=300)

In [ ]:
import plotly.graph_objects as go


def create_speedup_bar_chart(
    results,
    prefetchers,
    benchmarks,
    legend_title="Prefetcher",
    show_default=False,
):
    """
    Creates a grouped bar chart comparing speedup for different prefetchers.
    """
    fig = go.Figure()

    for key, props in prefetchers.items():
        if not show_default and key == "default":
            continue

        # Get the WP and CP configurations for this prefetcher
        wp_key = f"lcf_wp_{key}_df"
        cp_key = f"lcf_cp_{key}_df"

        # Calculate speedup: (WP-<> / CP-<>)
        wp_data = results[wp_key]["IPC"]
        cp_data = results[cp_key]["IPC"]
        speedup = ((wp_data / cp_data) - 1) * 100

        # Drop the "amean" benchmark if it exists
        if "amean" in speedup.index:
            speedup = speedup.drop("amean")

        # Add bar trace for the prefetcher
        fig.add_trace(
            go.Bar(
                x=benchmarks,
                y=speedup,
                name=props["label"],
                marker=dict(
                    color=props.get("color", None),  # Sets the fill color
                    line=dict(color=props.get("edgecolor", None)),  # Sets edge color
                    pattern_shape=props.get("hatch", None),  # Sets hatch pattern
                ),
            )
        )

    fig.update_layout(
        # title=title,
        xaxis=dict(
            title="Benchmark",
            tickangle=-45,
            # tickfont=dict(size=20),
            # titlefont=dict(size=24),
            showgrid=True,  # Show gridlines
            zeroline=True,  # Add a zero line
        ),
        yaxis=dict(
            title="Speedup (%)",
            # tickfont=dict(size=20),
            # titlefont=dict(size=24),
            showgrid=True,  # Show gridlines
            zeroline=True,  # Add a zero line
        ),
        # font=dict(size=24),
        autosize=True,
        height=750,
        # width=1200,
        barmode="group",
        # legend_title=legend_title,
        legend=dict(
            x=0.5,  # Center the legend horizontally
            y=0.15,  # Position the legend above the plot
            xanchor="right",  # Anchor the legend to the center horizontally
            yanchor="top",  # Anchor the legend to the bottom vertically
            orientation="h",  # Horizontal legend
        ),
        # plot_bgcolor="white",  # Remove background color
        # paper_bgcolor="white",  # Remove outer paper background color
        shapes=[
            # Add borders around the plot area (including x and y axes)
            dict(
                type="rect",
                x0=0,
                x1=1,
                y0=0,
                y1=1,
                xref="paper",
                yref="paper",
                line=dict(color="black"),  # Border color and width
            ),
        ],
    )
    return fig


# Define the L1I prefetchers and their properties
l1i_prefetchers = {
    "default": {"label": "FDIP", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l1i-barsa": {"label": "Barsa"},
    "l1i-bip": {"label": "BIP"},
    "l1i-djolt": {"label": "Djolt"},
    "l1i-epi": {"label": "EPI"},
    "l1i-fnlmma": {"label": "FNL-MMA"},
    "l1i-mana": {"label": "Mana"},
    "l1i-next_line": {"label": "Next Line"},
    "l1i-pips": {"label": "PIPS"},
    "l1i-tap": {"label": "TAP"},
}

# Define the benchmarks (these should match the indexes in the data)
benchmarks = results["lcf_wp_default_df"].index

if "amean" in benchmarks:
    benchmarks = benchmarks.drop("amean")

# Plot the speedup for the specified prefetchers
fig = create_speedup_bar_chart(
    results,
    l1i_prefetchers,
    benchmarks,
    # title="L1I Prefetcher Speedup Comparison",
    # legend_title="L1I Prefetcher",
    show_default=True,
)

# Show the plot
fig.show()

# Save the plot as an SVG file
fig.write_image("plot.svg", format="svg", width=1200)

### L1I, L1D, L2C, LLC TODO

### Instruction Footprint

In [ ]:
# Filter out 'gmean' from the DataFrame if it's in the index
cp_default_df_filtered = cp_default_df[cp_default_df.index != "gmean"]

# Convert both instr_foot_print and data_foot_print to KB for the filtered DataFrame
cp_instr_footprint = cp_default_df_filtered["instr_foot_print"] * 64 / 1024
cp_data_footprint = cp_default_df_filtered["data_foot_print"] * 64 / 1024

# Define the positions for the bars
x = np.arange(len(cp_default_df_filtered))  # The x locations for the groups
width = 0.35  # The width of the bars

# Create the plot
plt.figure(figsize=(14, 10))

# Plot both columns
plt.bar(
    x - width / 2,
    cp_instr_footprint,
    width,
    label="Instruction Footprint (KB)",
    color="skyblue",
)
plt.bar(
    x + width / 2, cp_data_footprint, width, label="Data Footprint (KB)", color="salmon"
)

# Set labels and title
plt.xlabel("Benchmark Names")
plt.ylabel("Footprint (KB)")
plt.title("Instruction and Data Footprint per Benchmark")

# Set the x-ticks to be the benchmark names, excluding 'gmean'
plt.xticks(x, cp_default_df_filtered.index, rotation=45, ha="right")

# Add a legend
plt.legend()

# Add grid lines for better readability
plt.grid(linestyle="--", alpha=0.7)

# Save the plot
plt.savefig(
    f"results/{suite}_footprint.svg", dpi=300, bbox_inches="tight", format="svg"
)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Compute instruction footprint in MB for CP and WP
cp_instr_footprint = cp_default_df["instr_foot_print"] * 64 / 1024**2
wp_instr_footprint = wp_default_df["instr_foot_print"] * 64 / 1024**2
wpa_instr_footprint = wpa_default_df["instr_foot_print"] * 64 / 1024**2

# Compute the percentage difference
percentage_diff_wp_cp = (
    (wp_instr_footprint - cp_instr_footprint) / cp_instr_footprint
) * 100

percentage_diff_wpa_cp = (
    (wpa_instr_footprint - cp_instr_footprint) / cp_instr_footprint
) * 100

# Drop gmean
percentage_diff_wp_cp = percentage_diff_wp_cp.drop("gmean")
percentage_diff_wpa_cp = percentage_diff_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=percentage_diff_wp_cp.index,  # Benchmarks on the x-axis
        y=percentage_diff_wp_cp,  # Percentage difference
        name="WP",
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=percentage_diff_wpa_cp.index,  # Benchmarks on the x-axis
        y=percentage_diff_wpa_cp,  # Percentage difference
        name="WPA",
        legendgroup="WPA",
    )
)

fig.update_layout(
    title="Instruction Footprint Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Instruction Footprint Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### IPC

In [ ]:
# Compute the IPC
cp_ipc = cp_default_df["IPC"]
wp_ipc = wp_default_df["IPC"]
wpa_ipc = wpa_default_df["IPC"]

# Drop amean
cp_ipc = cp_ipc.drop("amean")
wp_ipc = wp_ipc.drop("amean")
wpa_ipc = wpa_ipc.drop("amean")

# Calculate speedup
wp_speedup = ((wp_ipc / cp_ipc) - 1) * 100
wpa_speedup = ((wpa_ipc / cp_ipc) - 1) * 100

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=wp_speedup.index,  # Benchmarks on the x-axis
        y=wp_speedup,  # Speedup
        name="WP",  # Add legend name
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=wpa_speedup.index,  # Benchmarks on the x-axis
        y=wpa_speedup,  # Speedup
        name="WPA",  # Add legend name
        legendgroup="WPA",
    )
)

# Update layout for better readability
fig.update_layout(
    title="Speedup Due to Wrong Path (Aware) Support",
    yaxis_title="Speedup (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L1I Hits & Misses

In [ ]:
cp_l1i_hits = cp_default_df["L1I_TOTAL_HITS"]
wp_l1i_hits = wp_default_df["L1I_TOTAL_HITS"]
wpa_l1i_hits = wpa_default_df["L1I_TOTAL_HITS"]

cp_l1i_misses = cp_default_df["L1I_TOTAL_MISS"]
wp_l1i_misses = wp_default_df["L1I_TOTAL_MISS"]
wpa_l1i_misses = wpa_default_df["L1I_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l1i_hits_wp_cp = ((wp_l1i_hits - cp_l1i_hits) / cp_l1i_hits) * 100
l1i_misses_wp_cp = ((wp_l1i_misses - cp_l1i_misses) / cp_l1i_misses) * 100

l1i_hits_wpa_cp = ((wpa_l1i_hits - cp_l1i_hits) / cp_l1i_hits) * 100
l1i_misses_wpa_cp = ((wpa_l1i_misses - cp_l1i_misses) / cp_l1i_misses) * 100

# Drop gmean
l1i_hits_wp_cp = l1i_hits_wp_cp.drop("gmean")
l1i_misses_wp_cp = l1i_misses_wp_cp.drop("gmean")

l1i_hits_wpa_cp = l1i_hits_wpa_cp.drop("gmean")
l1i_misses_wpa_cp = l1i_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l1i_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l1i_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1i_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l1i_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1i_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L1I Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L1D Hits & Misses

In [ ]:
cp_l1d_hits = cp_default_df["L1D_TOTAL_HITS"]
wp_l1d_hits = wp_default_df["L1D_TOTAL_HITS"]
wpa_l1d_hits = wpa_default_df["L1D_TOTAL_HITS"]

cp_l1d_misses = cp_default_df["L1D_TOTAL_MISS"]
wp_l1d_misses = wp_default_df["L1D_TOTAL_MISS"]
wpa_l1d_misses = wpa_default_df["L1D_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l1d_hits_wp_cp = ((wp_l1d_hits - cp_l1d_hits) / cp_l1d_hits) * 100
l1d_misses_wp_cp = ((wp_l1d_misses - cp_l1d_misses) / cp_l1d_misses) * 100

l1d_hits_wpa_cp = ((wpa_l1d_hits - cp_l1d_hits) / cp_l1d_hits) * 100
l1d_misses_wpa_cp = ((wpa_l1d_misses - cp_l1d_misses) / cp_l1d_misses) * 100

# Drop gmean
l1d_hits_wp_cp = l1d_hits_wp_cp.drop("gmean")
l1d_misses_wp_cp = l1d_misses_wp_cp.drop("gmean")

l1d_hits_wpa_cp = l1d_hits_wpa_cp.drop("gmean")
l1d_misses_wpa_cp = l1d_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l1d_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l1d_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1d_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l1d_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1d_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L1D Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L2C Hits & Misses

In [ ]:
cp_l2c_hits = cp_default_df["L2C_TOTAL_HITS"]
wp_l2c_hits = wp_default_df["L2C_TOTAL_HITS"]
wpa_l2c_hits = wpa_default_df["L2C_TOTAL_HITS"]

cp_l2c_misses = cp_default_df["L2C_TOTAL_MISS"]
wp_l2c_misses = wp_default_df["L2C_TOTAL_MISS"]
wpa_l2c_misses = wpa_default_df["L2C_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l2c_hits_wp_cp = ((wp_l2c_hits - cp_l2c_hits) / cp_l2c_hits) * 100
l2c_misses_wp_cp = ((wp_l2c_misses - cp_l2c_misses) / cp_l2c_misses) * 100

l2c_hits_wpa_cp = ((wpa_l2c_hits - cp_l2c_hits) / cp_l2c_hits) * 100
l2c_misses_wpa_cp = ((wpa_l2c_misses - cp_l2c_misses) / cp_l2c_misses) * 100

# Drop gmean
l2c_hits_wp_cp = l2c_hits_wp_cp.drop("gmean")
l2c_misses_wp_cp = l2c_misses_wp_cp.drop("gmean")

l2c_hits_wpa_cp = l2c_hits_wpa_cp.drop("gmean")
l2c_misses_wpa_cp = l2c_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l2c_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l2c_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l2c_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l2c_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l2c_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L2C Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    # yaxis=dict(range=[0, 100]),  # Set the y-axis range from 0 to 100
)

# Show the plot
fig.show()

### LLC Hits & Misses

In [ ]:
cp_llc_hits = cp_default_df["LLC_TOTAL_HITS"]
wp_llc_hits = wp_default_df["LLC_TOTAL_HITS"]
wpa_llc_hits = wpa_default_df["LLC_TOTAL_HITS"]

cp_llc_misses = cp_default_df["LLC_TOTAL_MISS"]
wp_llc_misses = wp_default_df["LLC_TOTAL_MISS"]
wpa_llc_misses = wpa_default_df["LLC_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
llc_hits_wp_cp = ((wp_llc_hits - cp_llc_hits) / cp_llc_hits) * 100
llc_misses_wp_cp = ((wp_llc_misses - cp_llc_misses) / cp_llc_misses) * 100

llc_hits_wpa_cp = ((wpa_llc_hits - cp_llc_hits) / cp_llc_hits) * 100
llc_misses_wpa_cp = ((wpa_llc_misses - cp_llc_misses) / cp_llc_misses) * 100

# Drop gmean
llc_hits_wp_cp = llc_hits_wp_cp.drop("gmean")
llc_misses_wp_cp = llc_misses_wp_cp.drop("gmean")

llc_hits_wpa_cp = llc_hits_wpa_cp.drop("gmean")
llc_misses_wpa_cp = llc_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=llc_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=llc_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=llc_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=llc_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=llc_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="LLC Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    # yaxis=dict(range=[0, 100]),  # Set the y-axis range from 0 to 100
)

# Show the plot
fig.show()

In [ ]:
def all_in_one(cp_df, wp_df, wpa_df, prefetcher_list):
    # Correct Path
    df_cp_l1i_default = cp.group_by(cp_df, "default")
    df_cp_l1i_default = cp.calculate_means(df_cp_l1i_default)

    df_cp_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(cp_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_cp_l1i_prefetchers[key] = cp.calculate_speedup(
                df_cp_l1i_default, prefetcher
            )

    # Wrong Path
    df_wp_l1i_default = cp.group_by(wp_df, "default")
    df_wp_l1i_default = cp.calculate_means(df_wp_l1i_default)

    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(wp_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_wp_l1i_prefetchers[key] = cp.calculate_speedup(
                df_wp_l1i_default, prefetcher
            )

    # Wrong Path Aware
    df_wpa_l1i_default = cp.group_by(wpa_df, "default")
    df_wpa_l1i_default = cp.calculate_means(df_wpa_l1i_default)

    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(wpa_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
                df_wpa_l1i_default, prefetcher
            )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_cp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Correct Path (CP) | Speedup of each instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Wrong Path (WP) | Speedup of each instruction prefetcher compared to WP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Wrong Path Aware (WPA) | Speedup of each instruction prefetcher compared to WPA FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean(
        df_cp_l1i_prefetchers,
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of CP, WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
    ).show()

    # Correct Path
    df_cp_l1i_default = cp.group_by(cp_df, "default")
    df_cp_l1i_default = cp.calculate_means(df_cp_l1i_default)

    # Wrong Path
    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        prefetcher = cp.group_by(wp_df, key)
        prefetcher = cp.calculate_means(prefetcher)
        df_wp_l1i_prefetchers[key] = cp.calculate_speedup(df_cp_l1i_default, prefetcher)

    # Wrong Path Aware
    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        prefetcher = cp.group_by(wpa_df, key)
        prefetcher = cp.calculate_means(prefetcher)
        df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
            df_cp_l1i_default, prefetcher
        )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean_wp_wpa(
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
    ).show()

    # Wrong Path
    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        cp_prefetcher = cp.group_by(cp_df, key)
        wp_prefetcher = cp.group_by(wp_df, key)

        cp_prefetcher = cp.calculate_means(cp_prefetcher)
        wp_prefetcher = cp.calculate_means(wp_prefetcher)
        df_wp_l1i_prefetchers[key] = cp.calculate_speedup(cp_prefetcher, wp_prefetcher)

    # Wrong Path Aware
    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        cp_prefetcher = cp.group_by(cp_df, key)
        wpa_prefetcher = cp.group_by(wpa_df, key)

        cp_prefetcher = cp.calculate_means(cp_prefetcher)
        wpa_prefetcher = cp.calculate_means(wpa_prefetcher)
        df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
            cp_prefetcher, wpa_prefetcher
        )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP respective prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WPA instruction prefetcher compared to CP respective prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean_wp_wpa(
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
    ).show()

## L1I

In [ ]:
l1i_prefetchers = {
    "default": {"label": "FDIP", "color": "white", "edgecolor": "black", "hatch": "x"},
    "l1i-barsa": {"label": "Barsa", "color": "blue"},
    "l1i-bip": {"label": "BIP", "color": "green"},
    "l1i-djolt": {"label": "Djolt", "color": "orange"},
    "l1i-epi": {"label": "EPI", "color": "purple"},
    "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
    "l1i-fnlmma_new": {"label": "FNL-MMA New", "color": "cyan"},
    "l1i-mana": {"label": "Mana", "color": "magenta"},
    "l1i-next_line": {"label": "Next Line", "color": "yellow"},
    "l1i-pips": {"label": "PIPS", "color": "lime"},
    "l1i-tap": {"label": "TAP", "color": "brown"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l1i_prefetchers)

## L1D

In [ ]:
l1d_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l1d-berti": {"label": "Berti"},
    "l1d-ip_stride": {"label": "IP Stride"},
    "l1d-ipcp": {"label": "IPCP"},
    "l1d-next_line": {"label": "Next Line"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l1d_prefetchers)

## L2C

In [ ]:
l2c_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l2c-ampm": {"label": "AMPM"},
    "l2c-bingo": {"label": "Bingo"},
    "l2c-bop": {"label": "BOP"},
    "l2c-dspatch": {"label": "DSPatch"},
    "l2c-ip_stride": {"label": "IP Stride"},
    "l2c-ipcp": {"label": "IPCP"},
    "l2c-mlop": {"label": "MLOP"},
    "l2c-next_line": {"label": "Next Line"},
    "l2c-sandbox": {"label": "Sandbox"},
    "l2c-scooby": {"label": "Scooby"},
    "l2c-sms": {"label": "SMS"},
    "l2c-spp_dev": {"label": "SPP-Dev"},
    "l2c-spp_ppf_dev": {"label": "SPP-PPF-Dev"},
    "l2c-streamer": {"label": "Streamer"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l2c_prefetchers)

## LLC

In [ ]:
llc_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l2c-ip_stride": {"label": "IP Stride"},
    "l2c-next_line": {"label": "Next Line"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, llc_prefetchers)

## LLC Replacment Policy

In [ ]:
llc_replacement_policies = {
    "default": {"label": "LRU", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "ship": {"label": "SHIP"},
    "srrip": {"label": "SRRIP"},
    "drrip": {"label": "DRRIP"},
    "mockingjay": {"label": "Mockingjay"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, llc_replacement_policies)